In [49]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-and-real-news-dataset/True.csv
/kaggle/input/fake-and-real-news-dataset/Fake.csv


In [50]:
true_news=pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')
true_news['Rate']=0
true_news.head()

,title,text,subject,date,Rate
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [51]:
fake_news=pd.read_csv('/kaggle/input/fake-and-real-news-dataset/Fake.csv')
fake_news['Rate']=1
fake_news.head()

,title,text,subject,date,Rate
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [52]:
news=pd.merge(true_news,fake_news,how='outer',left_index=False,right_index=False)
news.head()

,title,text,subject,date,Rate
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


Model_Training

In [75]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
x=news['title']
y=news['Rate']
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0)
vect=CountVectorizer(max_df=5,ngram_range=(1,2)).fit(x_train)
names=np.array(vect.get_feature_names())
x_train_trans=vect.transform(x_train)

In [76]:
lr=LogisticRegression(C=100000).fit(x_train_trans,y_train)
predict=lr.predict(vect.transform(x_test))
coeff=lr.coef_[0].argsort()
smallest=names[coeff[:10]]
largest=names[coeff[:-11:-1]]

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Results

In [77]:
from sklearn.metrics import roc_auc_score,f1_score
print('Roc_Score {:.2%}'.format(roc_auc_score(y_test,predict)))
print('F1_Score{:.2%}'.format(f1_score(y_test,predict)))

Roc_Score 88.76%
F1_Score88.67%


In [106]:
pred_data['title'].values

array(['Etihad advises checks with U.S. missions after new Trump order',
       'Factbox: Trump U.S. Supreme Court pick could affect pending cases',
       'Mexico finance minister says NAFTA talks create economic uncertainty',
       ...,
       "U.S. attorney general calls threats against Jewish groups 'unacceptable'",
       "China sanguine as Trump-Xi 'bromance' sours over North Korea, Taiwan",
       'Charter schools see opportunity with Republicans in power: lobbying group'],
      dtype=object)

In [111]:
lr.predict(vect.transform(['Etihad advises checks with U.S. missions after new Trump order']))

array([0])